# In this notebook, we have the code to test a GAN for the multi minst data

In [1]:
import torch
import torch.optim as optim
from dataloaders import get_mnist_dataloaders, get_multi_mnist_dataloaders
from models import Generator, Discriminator
from training import Trainer
from torch.utils.data import Dataset
import h5py
import numpy as np
from torchvision.utils import make_grid
from torch.autograd import Variable
import matplotlib.pyplot as plt
import os
from PIL import Image

In [2]:
import netdissect.actviz as act

In [29]:
# Load models
generator = torch.load('gen_mnist_model_epoch_190.pt',map_location='cpu')
discriminator = torch.load('dis_mnist_model_epoch_190.pt',map_location='cpu')
print(generator)
for name, param in generator.state_dict().items():
    print(name[18])
img_size = (32, 32, 1)

C:\Users\Lucy\Anaconda3\lib\site-packages\torch\serialization.py:435: SourceChangeWarning: source code of class 'models.Generator' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Generator(
  (latent_to_features): Sequential(
    (0): Linear(in_features=100, out_features=512, bias=True)
    (1): ReLU()
  )
  (features_to_image): Sequential(
    (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ConvTranspose2d(32, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ConvTranspose2d(16, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
    (10): Sigmoid()
  )
)
.
.
0
0
2
2
2
2
2
3
3
5
5
5
5
5
6
6
8
8
8
8
8
9
9


In [75]:
plt.figure(figsize=(50,8))
# fixed_latents = Variable(torch.rand(10, 32, 100))
for ten in range(10):
    fixed_latents[ten] = Variable(generator.sample_latent(32))
#fixed_latents = Variable(generator.sample_latent(32))
name_lst = ['conv1', 'relu1', 'bn1', 'conv2', 'relu2', 'bn2', 'conv3', 'relu3', 'bn3', 'conv4', 'sig4']


# for i in range(len(name_lst)):
#     nm = name_lst[i]  # loop over each layers
#     if not os.path.exists("img/{}".format(nm)):  # make dir for that layers
#         os.mkdir("img/{}".format(nm))

#     img, latent = generator(fixed_latents)
#     ll = latent[i]  # inspect this layer i
#     unit_num = ll.shape[1]  # this layer has this many unit
#     for un in range(unit_num):
#         for count in range(10):
#             plt.subplot(1,10,count+1)
            
#             # Generate batch of images and convert to grid
#             img, latent = generator(fixed_latents[count])
#             l = latent[i]
#             result = act.activation_visualization(img[0][0].detach().numpy(), l[0][un].detach().numpy(), level=np.percentile(l[0][un].detach().numpy(),95))
#             plt.imshow(result)

#         plt.savefig("img/{}/{}-with.jpg".format(nm, un), bbox_inches='tight', pad_inches=0)


pname_lst = []
import copy
origin_generator = copy.deepcopy(generator)

ori_img_lst = []
ori_latent_lst = []
for ct in range(10):
    ori_img, ori_latent = origin_generator(fixed_latents[count])
    ori_img_lst.append(ori_img)
    ori_latent_lst.append(ori_latent)

for pname, param in generator.state_dict().items():
    pname_lst.append(pname)
#     print(generator.state_dict()[pname])

idx_lst = [9, 11, 16, 18, 23]
from operator import itemgetter

pname_lst = itemgetter(*idx_lst)(pname_lst)
print(pname_lst)
for j in pname_lst:
    
    idx = j[18]
    print(idx)
    nm = name_lst[int(idx)]
    print(nm)

    if not os.path.exists("img/{}".format(nm)):  # make dir for that layers
        os.mkdir("img/{}".format(nm))

    img, latent = generator(fixed_latents[0])
    ll = latent[int(idx)]  # inspect this layer i
    unit_num = ll.shape[1]  # this layer has this many unit
    print(unit_num)
    iou_ave = 0
    for un in range(unit_num):
        for count in range(10):
            plt.subplot(1,10,count+1)
            
            # Generate batch of images and convert to grid
            generator = copy.deepcopy(origin_generator)
            for pnm in pname_lst:
                print(pnm)
                if pnm[18] == idx:
                    print(generator.state_dict()[pnm].shape)
                    if len(generator.state_dict()[pnm].shape) == 4:  # put weight to be zeros: (128, 64, x, x)
                        generator.state_dict()[pnm][:, un, ...] = torch.zeros_like(generator.state_dict()[pnm][:, un, ...])
                    elif len(generator.state_dict()[pnm].shape) == 1:  # put bias to be zeros: (64,)
                        generator.state_dict()[pnm][un] = 0
                    else:
                        print(generator.state_dict()[pnm].shape)
            
            img, latent = generator(fixed_latents[count])
            iou = torch.norm(ori_img_lst[count] - img)
            iou_ave += iou
            l = latent[int(idx)]
            result = act.activation_visualization(img[0][0].detach().numpy(), l[0][un].detach().numpy(), level=np.percentile(l[0][un].detach().numpy(),80))
            plt.imshow(result)

        iou_ave /= 10
        plt.savefig("img/{}/{}-without-{:.3f}.jpg".format(nm, un, iou_ave), bbox_inches='tight', pad_inches=0)

C:\Users\Lucy\Anaconda3\lib\site-packages\matplotlib\pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


('features_to_image.3.weight', 'features_to_image.5.weight', 'features_to_image.6.weight', 'features_to_image.8.weight', 'features_to_image.9.weight')
3
conv2
32
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
t

C:\Users\Lucy\Anaconda3\lib\site-packages\matplotlib\figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
torch.Size([64, 32, 4, 4])
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
features_to_image.3.weight
t

features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight


...

features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
torch.Size([16])
features_to_image.9.weight


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
torch.Size([16])
features_to_image.9.weight


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


9
conv4
1
features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
torch.Size([16, 1, 4, 4])
features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
torch.Size([16, 1, 4, 4])
features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
torch.Size([16, 1, 4, 4])
features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
torch.Size([16, 1, 4, 4])
features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
torch.Size([16, 1, 4, 4])
features_to_image.3.weight
features_to_image.5.weight
features_to_image.6.weight
features_to_image.8.weight
features_to_image.9.weight
torch.Size([16, 1, 4, 4])
features_to_image.3.weig